# Experimental FLD scoring 

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

from peakqc.fld_scoring import *
import peakqc.insertsizes as ins

In [2]:
fragments_file = "/mnt/workspace2/jdetlef/data/public_data/fragments_heart_left_ventricle_194_sorted.bed"
h5ad_file = "/mnt/workspace2/jdetlef/data/public_data/heart_lv_SM-JF1NY.h5ad"

In [3]:
adata = sc.read_h5ad(h5ad_file)
adata_barcodes = adata.obs.index.tolist()

# split index for barcodes CBs
barcodes = []
for entry in adata_barcodes:
    barcode = entry.split('+')[1]
    barcodes.append(barcode)

In [ ]:
count_table_ori = ins.insertsize_from_fragments(fragments=fragments_file,
                              barcodes=barcodes,
                              n_threads=8)

count_table = count_table_ori.copy()

Count insertsizes from fragments...
Starting counting fragments...


Processing Chunks: 17it [01:11,  4.19s/it]


In [ ]:
count_table = count_table_ori.copy()

In [ ]:
means = count_table.pop('mean_insertsize')
insert_counts = count_table.pop('insertsize_count')
barcodes = count_table.index
# Assuming count_table is your DataFrame and you're trying to convert 'dist' column to a numpy array of type int64
dists_arr = np.array(count_table['dist'].tolist(), dtype=np.int64)
dists_arr

In [ ]:
plot = True
save_plots = 'density_plot'
# plot the densityplot of the fragment length distribution
if plot:
    print("plotting density...")
    density_plot(dists_arr, max_abundance=600, save=save_plots)

In [ ]:
wavelength = 150
sigma = 0.4
plot=True
n_threads=8
peaks_thr=1
save_overview='ov_test'
sample=0


print("calculating scores using the custom continues wavelet transformation...")
conv_scores = score_by_conv(data=dists_arr,
                            wavelength=wavelength,
                            sigma=sigma,
                            plot_wavl=plot,
                            n_threads=n_threads,
                            peaks_thr=peaks_thr,
                            operator='bigger',
                            plot_mask=plot,
                            plot_ov=plot,
                            save=save_overview,
                            sample=sample)